In [8]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.common.keys import Keys
import time
from datetime import datetime, timedelta

In [ ]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime, timedelta
import time

# GeckoDriver yolu Service üzerinden belirtiliyor
service = Service(r'C:\Users\furka\Downloads\geckodriver-v0.35.0-win64\geckodriver.exe')

# WebDriver'ın başlatılması (Mozilla Firefox kullanılıyor)
driver = webdriver.Firefox(service=service)

# TEFAS sayfasını aç
driver.get('https://www.tefas.gov.tr/TarihselVeriler.aspx')

# Tarih aralığı başlangıcı (bugünün tarihi)
end_date = datetime.today()

# Kaç ay geri gideceğimizi belirtelim
months_back = 3

# Yeni CSV butonunun XPath'i
csv_button_xpath = '/html/body/form/div[3]/div[3]/div[2]/div[1]/div[2]/div[1]/button[4]'

# Fon ismi kutusunun XPath'i
fund_input_xpath = '//*[@id="TextBoxOtherFund"]'

def set_date_range(start_date, end_date):
    # Yeni XPath'lerle tarih alanlarını bul ve ayarla
    start_date_input = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="TextBoxStartDate"]'))
    )
    end_date_input = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="TextBoxEndDate"]'))
    )

    start_date_input.clear()
    start_date_input.send_keys(start_date.strftime('%d.%m.%Y'))
    
    end_date_input.clear()
    end_date_input.send_keys(end_date.strftime('%d.%m.%Y'))

def set_fund(fund_name):
    # Fon ismini gir
    fund_input = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, fund_input_xpath))
    )
    fund_input.clear()
    fund_input.send_keys(fund_name)

    # Fon ismi girildikten sonra, açılan listeden fonu seç
    # Beklemeyi artırarak ve JavaScript ile sayfa kaydırma ekleyerek çözebiliriz
    try:
        time.sleep(2)  # Açılan listenin yüklenmesi için küçük bir bekleme
        dropdown = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="MainContent_AutoCompleteExtenderOtherFund_completionListElem"]/ul/li'))
        )
        driver.execute_script("arguments[0].scrollIntoView(true);", dropdown)
        dropdown.click()
    except TimeoutException as e:
        print(f"Dropdown listesinde fon bulunamadı: {str(e)}")

# Fon arama işlemini başlat
def search_funds():
    search_button = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.ID, 'ctl00_ContentPlaceHolder1_Button1'))
    )
    search_button.click()

# CSV indir
def download_csv():
    try:
        WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, csv_button_xpath))).click()
        time.sleep(10)  # İndirmenin tamamlanmasını bekle
    except (NoSuchElementException, TimeoutException):
        print(f"Veri bulunamadı veya CSV indirilemedi. Tarih aralığı: {start_date.strftime('%d.%m.%Y')} - {end_date.strftime('%d.%m.%Y')}")
        return False  # Eğer veri yoksa veya CSV butonuna tıklanamadıysa False döndür

    return True  # Eğer CSV başarıyla indirildiyse True döndür

# 3 ay geriye giderek CSV'leri indir
fund_name = 'KPC - KUVEYT TÜRK PORTFÖY KATILIM HİSSE SENEDİ (TL) FONU (HİSSE SENEDİ YOĞUN FON)'  # Burada istediğin fonun ismini belirle

while end_date.year > 2000:  # 2000 yılını örnek verdim, istediğin tarihe kadar gidilebilir
    start_date = end_date - timedelta(days=months_back * 30)

    set_fund(fund_name)
    set_date_range(start_date, end_date)
    search_funds()
    if not download_csv():  # Eğer download_csv() False dönerse döngüyü kır
        break
    
    # Bir önceki döneme geç
    end_date = start_date - timedelta(days=1)

# İşlem bittiğinde tarayıcıyı kapat
driver.quit()


Dropdown listesinde fon bulunamadı: Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:511:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

